# EDA i Feature Engineering - część 2
- Wczytanie danych tras karetek z pliku Parquet, ustawienie odpowiedniego układu współrzędnych (CRS)
- Porównanie współrzędnych referencyjnych z początkiem i końcem każdej trasy w celu sprawdzenia ich zgodności
- Wykrywanie zagęszczeń punktów (potencjalnych postojów) w środkowej części trasy za pomocą algorytmu opartego na odległości
- Wyznaczenie dla każdego wykrytego skupiska punktów reprezentatywnych pokrywających obszar klastra
- Dodanie nowych kolumn z informacją o klastrach i punktach reprezentatywnych
- Konwersja list obiektów Point do list tupli na potrzeby zapisu do Parquet
- Wizualizacja wybranych tras z zaznaczeniem startu i końca trasy GPS, punktu referencyjnego ze zgłoszenia i wykrytych skupisk na mapie

In [1]:
import geopandas as gpd
import branca.colormap as cm
import folium
import numpy as np
from shapely.geometry import Point
import swifter

# ! pip install swifter tqdm
data_path = r"X:\dane_karetki\karetki.parquet"

In [2]:
if 'gdf' not in globals():
    # automatycznie odczyta geometrię
    gdf = gpd.read_parquet(data_path)

    gdf = gdf.sort_values(by='Czas wezwania', ascending=True)

    # Próbkowanie do 10% danych
    # gdf = gdf.sample(frac=0.1, random_state=42) # Dodaj random_state dla powtarzalności
    gdf = gdf.reset_index(drop=True)

    if gdf.crs is None:
         gdf = gdf.set_crs(4326, allow_override=True)
    elif gdf.crs.to_epsg() != 4326:
         gdf = gdf.to_crs(4326)

In [3]:
none_lines_count = gdf['Line'].isnull().sum()
print(f"Liczba wierszy z None w kolumnie 'Line': {none_lines_count}")

Liczba wierszy z None w kolumnie 'Line': 559


In [4]:
gdf = gdf.dropna(subset=['Line']).copy()

In [5]:
gdf.head()

,Czas wezwania,Czas wyjazdu ZRM,Czas powrotu ZRM,Powód wezwania,Kod pilności,Dlugość geograficzna,Szerokość geograficzna,Identyfikator pojazdu,"Rodzaj wyjazdu 0- na sygnale, 1 -zwykly",Typ zespolu,"Określenie wieku pacjenta 0- dziecko, 1 - dorosly",ID_GPS,Line,Line_time,Line_points,Line_unique_points
0,2021-03-31 22:11:24,2021-03-31 22:26:13,2021-03-31 22:59:00,Inne,1,19.848080,50.086277,KR 4F001,1,P,1,181375,"LINESTRING (19.92509 50.07631, 19.92509 50.076...","[2021-03-31 22:26:26, 2021-03-31 22:26:41, 202...",205,32
1,2021-03-31 22:34:53,2021-03-31 22:37:25,2021-04-01 00:05:08,Duszność,1,19.803841,49.516422,KNT TU09,0,P,0,173168,"LINESTRING (19.8756 49.56501, 19.8756 49.56501...","[2021-03-31 22:37:31, 2021-03-31 22:37:46, 202...",370,34
2,2021-03-31 22:38:57,2021-03-31 22:43:46,2021-04-01 00:09:52,Ból brzucha,1,19.895836,50.078758,KR 5Y997,1,P,1,250824,"LINESTRING (19.88432 50.08181, 19.88432 50.081...","[2021-03-31 22:43:48, 2021-03-31 22:44:03, 202...",391,65
3,2021-03-31 22:50:12,2021-03-31 22:55:15,2021-04-01 00:30:46,Problemy kardiologiczne,1,20.033554,50.072250,KR 5ST66,0,P,1,74302,"LINESTRING (20.01848 50.09211, 20.01848 50.092...","[2021-03-31 22:55:23, 2021-03-31 22:55:38, 202...",410,16
4,2021-03-31 22:51:59,2021-03-31 22:53:34,2021-03-31 23:07:34,Leży,1,19.566053,50.279728,KR 6MY69,0,S,1,410286,"LINESTRING (19.5614 50.26539, 19.5614 50.26539...","[2021-03-31 22:53:53, 2021-03-31 22:54:53, 202...",54,40


In [6]:
def plot_route(row, cluster_info):
    # Ensure geometry is in EPSG:4326
    line = row['Line']
    if hasattr(line, 'crs'):
        # If line is a GeoSeries, ensure correct CRS
        if getattr(line, 'crs', None) != 4326:
            line = gpd.GeoSeries([line], crs=line.crs).to_crs(4326).iloc[0]
    else:
        # If not, assume gdf.crs and convert
        line = gpd.GeoSeries([line], crs=gdf.crs).to_crs(4326).iloc[0]
    coords = [(y, x) for x, y in line.coords]
    colormap = cm.linear.viridis.scale(0, len(coords) - 2)
    m = folium.Map(location=coords[0], zoom_start=13)

    for i in range(len(coords) - 1):
        segment = [coords[i], coords[i + 1]]
        color = colormap(i)
        folium.PolyLine(segment, color=color, weight=5).add_to(m)

    folium.Marker(location=coords[0], popup='Start', icon=folium.Icon(color='green', icon='play')).add_to(m)
    folium.Marker(location=coords[-1], popup='End', icon=folium.Icon(color='red', icon='stop')).add_to(m)

    # Convert reference point to EPSG:4326 if needed
    lat = row['Szerokość geograficzna']
    lon = row['Dlugość geograficzna']
    folium.Marker(location=[lat, lon], popup='Extra point', icon=folium.Icon(color='blue', icon='info-sign')).add_to(m)

    has_cluster, dense_points = cluster_info
    if has_cluster and len(dense_points) > 0:
        dense_points = [Point(pt) for pt in dense_points]
        dense_points_gs = gpd.GeoSeries(dense_points, crs=gdf.crs).to_crs(4326)
        for pt in dense_points_gs:
            folium.CircleMarker(
                location=[pt.y, pt.x],
                radius=6,
                color='orange',
                fill=True,
                fill_color='orange',
                fill_opacity=0.7,
                popup='Dense cluster'
            ).add_to(m)

    colormap.caption = 'Order of points'
    colormap.add_to(m)
    return m

#### Z jakiegoś powodu zostały jeszcze trasy ze wszystkimi punktami w odległości 200m od startu - usuwamy je jeszcze raz

In [7]:
gdf = gdf.to_crs(3857)

def all_points_within_m_proj(line, threshold=200):
    coords = np.array(line.coords)
    start = coords[0]
    dists = np.linalg.norm(coords - start, axis=1)
    return np.all(dists <= threshold)

In [8]:
result = gdf['Line'].apply(all_points_within_m_proj)
round(float(result.sum() / len(gdf) * 100), 2)

2.09

In [9]:
gdf = gdf[~result].copy()

## Sprawdzenie, czy koordynaty w kolumnach 'Szerokość geograficzna' i 'Dlugość geograficzna' są zgodne z początkami/końcami linestringów

#### Porównanie z końcami linestringów

In [10]:
# Vectorized extraction of last points from LineString
last_points = gdf['Line'].apply(lambda line: Point(line.coords[-1]))
last_points_gs = gpd.GeoSeries(last_points, crs=3857).reset_index(drop=True)

ref_points = gpd.GeoSeries(
    [Point(lon, lat) for lon, lat in zip(gdf['Dlugość geograficzna'], gdf['Szerokość geograficzna'])],
    crs=4326
).to_crs(3857).reset_index(drop=True)

distances = last_points_gs.distance(ref_points, align=True)
coord_within_200m_of_last = distances <= 200
float((coord_within_200m_of_last.sum() / len(gdf)) * 100)

0.5369944011302391

#### Porównanie z początkami linestringów

In [11]:
first_points = gdf['Line'].apply(lambda line: Point(line.coords[0]))
first_points_gs = gpd.GeoSeries(last_points, crs=3857).reset_index(drop=True)

distances = first_points_gs.distance(ref_points, align=True)
coord_within_200m_of_first = distances <= 500
float((coord_within_200m_of_first.sum() / len(gdf)) * 100)

1.754879388833656

## Sprawdzenie zdublowanych punktów w środku trasy (postój)

In [12]:
def has_dense_cluster(line, min_neighbors=10, distance_threshold=40, return_points=False):
    coords = np.array(line.coords)
    n = len(coords)
    if n < 5:
        return (False, np.array([])) if return_points else False

    start_idx = int(n * 0.25)
    end_idx = int(n * 0.75)
    middle_coords = coords[start_idx:end_idx]
    if len(middle_coords) < min_neighbors + 1:
        return (False, np.array([])) if return_points else False

    dists = np.linalg.norm(middle_coords[:, None, :] - middle_coords[None, :, :], axis=2)
    neighbor_counts = (dists < distance_threshold).sum(axis=1) - 1
    mask = neighbor_counts >= min_neighbors
    has_cluster = np.any(mask)
    dense_points = middle_coords[mask]
    if return_points:
        return has_cluster, dense_points
    else:
        return has_cluster

In [17]:
results = gdf['Line'] \
    .swifter.apply(lambda line: has_dense_cluster(line, return_points=True))

Dask Apply:   0%|          | 0/16 [00:00<?, ?it/s]

In [18]:
gdf['has_cluster'], gdf['dense_points'] = zip(*results)

In [19]:
random = gdf.sample(1)
info = has_dense_cluster(random.iloc[0]['Line'], return_points=True)
info[0]

False

In [20]:
display(plot_route(random.iloc[0], info))

In [21]:
random

,Czas wezwania,Czas wyjazdu ZRM,Czas powrotu ZRM,Powód wezwania,Kod pilności,Dlugość geograficzna,Szerokość geograficzna,Identyfikator pojazdu,"Rodzaj wyjazdu 0- na sygnale, 1 -zwykly",Typ zespolu,"Określenie wieku pacjenta 0- dziecko, 1 - dorosly",ID_GPS,Line,Line_time,Line_points,Line_unique_points,has_cluster,dense_points
151294,2022-08-27 10:13:12,2022-08-27 10:15:21,2022-08-27 10:53:31,Zaburzenia psychiczne,1,19.888304,50.006496,KR 9VJ49,1,P,1,141505,"LINESTRING (2218321.463 6457313.174, 2218321.4...","[2022-08-27 10:15:52, 2022-08-27 10:16:52, 202...",208,185,False,[]


## Policzenie punktów reprezentatywnych dla każdego skupienia i dodanie nowej kolumny

In [22]:
def simple_dense_representatives(dense_points, distance_threshold=40):
    if dense_points is None or len(dense_points) == 0:
        return None

    points = np.array(dense_points)
    n_points = len(points)
    is_covered = np.zeros(n_points, dtype=bool)
    representatives = []

    while np.any(~is_covered):
        first_uncovered_idx = np.where(~is_covered)[0][0]
        current_rep = points[first_uncovered_idx]
        representatives.append(Point(current_rep))

        distances = np.linalg.norm(points - current_rep, axis=1)
        nearby_points_mask = distances < distance_threshold

        is_covered[nearby_points_mask] = True

    return representatives if representatives else None

In [23]:
gdf['dense_cluster'] = gdf['dense_points'].swifter.apply(simple_dense_representatives)

Pandas Apply:   0%|          | 0/152888 [00:00<?, ?it/s]

In [24]:
gdf.head()

,Czas wezwania,Czas wyjazdu ZRM,Czas powrotu ZRM,Powód wezwania,Kod pilności,Dlugość geograficzna,Szerokość geograficzna,Identyfikator pojazdu,"Rodzaj wyjazdu 0- na sygnale, 1 -zwykly",Typ zespolu,"Określenie wieku pacjenta 0- dziecko, 1 - dorosly",ID_GPS,Line,Line_time,Line_points,Line_unique_points,has_cluster,dense_points,dense_cluster
0,2021-03-31 22:11:24,2021-03-31 22:26:13,2021-03-31 22:59:00,Inne,1,19.848080,50.086277,KR 4F001,1,P,1,181375,"LINESTRING (2218050.748 6459501.092, 2218050.7...","[2021-03-31 22:26:26, 2021-03-31 22:26:41, 202...",205,32,True,"[[2219066.299255437, 6459487.858454758], [2219...",[POINT (2219066.299255437 6459487.858454758)]
2,2021-03-31 22:38:57,2021-03-31 22:43:46,2021-04-01 00:09:52,Ból brzucha,1,19.895836,50.078758,KR 5Y997,1,P,1,250824,"LINESTRING (2213512.3 6460455.303, 2213512.3 6...","[2021-03-31 22:43:48, 2021-03-31 22:44:03, 202...",391,65,True,"[[2213476.629237005, 6460375.89129967], [22134...","[POINT (2213476.629237005 6460375.89129967), P..."
4,2021-03-31 22:51:59,2021-03-31 22:53:34,2021-03-31 23:07:34,Leży,1,19.566053,50.279728,KR 6MY69,0,S,1,410286,"LINESTRING (2177565.239 6492364.604, 2177565.2...","[2021-03-31 22:53:53, 2021-03-31 22:54:53, 202...",54,40,False,[],None
5,2021-03-31 22:54:20,2021-03-31 22:55:48,2021-04-01 00:46:00,Zaburzenia psychiczne,2,20.720327,49.460987,KN 92599,1,P,1,168320,"LINESTRING (2296759.443 6370829.534, 2296759.4...","[2021-03-31 22:55:57, 2021-03-31 22:56:12, 202...",432,290,True,"[[2296749.8880768977, 6370851.138941719], [229...","[POINT (2296749.8880768977 6370851.138941719),..."
6,2021-03-31 23:07:41,2021-03-31 23:11:23,2021-03-31 23:46:00,Zasłabnięcie,1,19.940767,50.052273,KR 5ST67,1,P,1,124191,"LINESTRING (2220967.033 6456783.979, 2220967.0...","[2021-03-31 23:11:30, 2021-03-31 23:11:45, 202...",172,66,True,"[[2223945.741886053, 6461909.327726552], [2223...",[POINT (2223945.741886053 6461909.327726552)]


In [25]:
gdf = gdf.drop(["dense_points", "Line_points", "Line_unique_points"], axis=1)

Parquet nie potrafi interpretować listy obiektów Point, bo to object, więc zmieniamy na listę tupli, po wczytaniu tych danych musimy zastosować zakomentowany kod z ostatniej komórki

In [26]:
gdf[gdf.select_dtypes('category').columns] = gdf.select_dtypes('category').astype(str)

In [27]:
def to_tuples(points):
    if points is None:
        return None
    return [(p.x, p.y) for p in points]

gdf['dense_cluster_xy'] = gdf['dense_cluster'].apply(to_tuples)
gdf.drop(columns=['dense_cluster']) \
    .to_parquet(r"X:\dane_karetki\karetki_klastry.parquet", engine='pyarrow', index=False)


In [28]:
# gdf['dense_cluster'] = gdf['dense_cluster_xy'].apply(
#     lambda lst: [Point(xy) for xy in lst] if lst is not None else None
# )